In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
from xgboost import XGBRegressor, XGBRFRegressor 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.decomposition import PCA

%matplotlib notebook

In [5]:
X_TRAIN_PATH = './Case_material/train/X_final.csv'
Y_TRAIN_PATH = './Case_material/train/y_train.csv'

X = pd.read_csv(X_TRAIN_PATH)
y = pd.read_csv(Y_TRAIN_PATH)

X.drop(columns=['ValueDateTimeUTC', 'time'], inplace=True)
# X.drop(columns=['ValueDateTimeUTC'], inplace=True)
y.drop(columns=['ValueDateTimeUTC'], inplace=True)

# pca = PCA(n_components=25)
# pca.fit(X)
# X = pca.transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [12]:
class XGB_per_hour:
    def __init__(self):
        self.models = list()
        for i in range(24):
            xgb_model = XGBRegressor()
            self.models.append(xgb_model)
        
    def fit(self, X_train, y_train):
        for i in range(24):
            print("Fitting model ", i)
            hX_train = X_train.loc[X_train['hour'] == i]
            hy_train = y_train.loc[X_train['hour'] == i]
            self.models[i].fit(hX_train, hy_train)
    
    def predict(self, X_test):        
        all_preds = pd.DataFrame()
        orig_indices = X_test.index.values
        for i in range(24):
            df_slice = X_test[X_test['hour'] == i]
            indices = X_test[X_test['hour'] == i].index.values
            
            assert(df_slice.shape[0] == indices.shape[0])
            
            preds = pd.DataFrame(self.models[i].predict(df_slice), index=indices)
            all_preds.iloc[indices] = preds
#             all_preds = all_preds.append(preds)
            
#         print(all_preds.iloc[[orig_indices]])
        return all_preds

    def predict_iter(self, X_test):
        def _predict_model(row):
            row_df = pd.DataFrame(row)
            row_df = row_df.transpose()
            return self.models[int(row['hour'])].predict(row_df)
            
#         preds = X_test.copy()
        x_copy = pd.DataFrame(X_test.copy())
        res = pd.DataFrame()
        res["Y"] = x_copy.apply(lambda row : _predict_model(row), axis=1)
        
        return res["Y"]
        
#         for (index, row) in X_test.iterrows():
#             print(index)
#             print(row)
# #             print(row.values.reshape(1,-1))
#             row.values.reshape(-1)
#             temp_row = pd.DataFrame(row, index=[index])
            
#             print(temp_row)
            
#             preds = self.models[1].predict(temp_row)
# #             print(temp_df)
#             print(preds)
#             break
            
#         return preds
    
    
    def evaluate(self, X_test, y_test):        
        mses = []
        mapes = []
        mape_accs = []

        for i in range(24):
            print("Evaluating Model ", i)
            hX_test = X_test.loc[X_test['hour'] == i]
            hy_test = y_test.loc[X_test['hour'] == i]
            
            h_preds = self.models[i].predict(hX_test)

            mse = round(mean_squared_error(hy_test.to_numpy(), h_preds.reshape((-1, 1))),2)
            mape = np.mean(np.abs((hy_test.to_numpy() - h_preds.reshape((-1,1))) / np.abs(hy_test.to_numpy())))
            mape_acc = round(100*(1 - mape), 2)

            mses.append(mse)
            mapes.append(mape)
            mape_accs.append(mape_acc)
            
        return np.mean(mses), np.mean(mapes), np.mean(mape_accs)

In [13]:
xgb_ph = XGB_per_hour()

xgb_ph.fit(X_train, y_train)

Fitting model  0
Fitting model  1
Fitting model  2
Fitting model  3
Fitting model  4
Fitting model  5
Fitting model  6
Fitting model  7
Fitting model  8
Fitting model  9
Fitting model  10
Fitting model  11
Fitting model  12
Fitting model  13
Fitting model  14
Fitting model  15
Fitting model  16
Fitting model  17
Fitting model  18
Fitting model  19
Fitting model  20
Fitting model  21
Fitting model  22
Fitting model  23


In [ ]:
preds = xgb_ph.predict_iter(X_test)

print(preds.shape)
print(X_test.shape)

# print(preds.iloc[[X_test.index.values]])
print(preds.head())
print(y_test.head())

# # preds = preds.to_numpy()
mse = round(mean_squared_error(y_test.to_numpy(), preds.to_numpy()),2)
mape = np.mean(np.abs((y_test.to_numpy() - preds.to_numpy())) / np.abs(y_test.to_numpy()))
mape_acc = round(100*(1 - mape), 2)

print("\n===================")
print("MSE: ", mse)
print("MAPE: ", mape)
print("Acc.: ", mape_acc)

In [ ]:
mse, mape, acc = xgb_ph.evaluate(X_test, y_test)

print("\n===================")
print("MSE: ", mse)
print("MAPE: ", mape)
print("Acc.: ", acc)

In [250]:
preds = pd.DataFrame()
X_np = X_test.to_numpy()

print(X_np)
for index in X_np:
    print(index)
#     temp_row = pd.DataFrame(row, index=[index])

#     print(temp_row)

[[0.21317985 0.1971634  0.04375233 ... 0.         9.         0.        ]
 [0.2552433  0.18922165 0.03853944 ... 0.         9.         0.        ]
 [0.23756347 0.13012158 0.23703564 ... 0.         1.         1.        ]
 ...
 [0.19128522 0.09141401 0.18497972 ... 2.         3.         0.        ]
 [0.19481322 0.18595291 0.08228974 ... 0.         4.         2.        ]
 [0.20777306 0.32485436 0.08667209 ... 0.         9.         1.        ]]
[2.13179847e-01 1.97163397e-01 4.37523292e-02 8.49958068e-02
 1.17495239e-02 0.00000000e+00 2.32711623e-01 1.48897596e-01
 2.95132173e-02 2.18627312e-01 1.79112568e-01 4.16327943e-03
 1.33528579e-01 8.54333215e-02 3.16684151e-02 1.06032318e-01
 7.48138836e-02 1.22518831e-04 8.39319190e-02 6.39187197e-02
 1.65725820e-02 8.85903716e-02 0.00000000e+00 0.00000000e+00
 3.47770601e-02 2.43900655e-02 7.74404046e-03 2.34508462e-02
 2.01500340e-02 6.63756972e-05 2.87355490e-02 1.95239103e-02
 2.52587979e-03 1.99741331e-02 3.47307336e-06 0.00000000e+00
 2.2000

[2.32144622e-01 1.35120991e-01 1.81835658e-01 5.77391652e-02
 3.54289667e-01 2.14010375e-02 2.29723056e-01 1.35580394e-01
 1.00285902e-01 1.97409759e-01 3.00959166e-01 2.34509501e-02
 1.13448991e-01 6.86781616e-02 4.47270117e-02 9.57128474e-02
 1.31465388e-01 7.06191413e-03 8.42157075e-02 5.10494854e-02
 5.79440135e-02 8.03477898e-02 8.82542223e-02 2.47877754e-03
 3.13133816e-02 2.29202672e-02 1.83934496e-02 1.93932408e-02
 2.94920541e-02 2.05765222e-04 2.45934853e-02 1.42324957e-02
 2.16805832e-02 1.79855622e-02 2.51036310e-02 8.42048789e-03
 1.20000000e+01 0.00000000e+00 5.00000000e+00 2.00000000e+00]
[1.71242713e-01 1.36696211e-01 1.60379438e-02 1.33929387e-01
 0.00000000e+00 0.00000000e+00 1.66850550e-01 1.09464650e-01
 3.76465362e-02 2.56723985e-01 6.17458467e-02 0.00000000e+00
 8.70861014e-02 6.48976121e-02 1.84105942e-02 1.26959139e-01
 0.00000000e+00 0.00000000e+00 6.36816205e-02 6.05544161e-02
 2.42421744e-02 1.02544144e-01 0.00000000e+00 0.00000000e+00
 2.24429216e-02 1.86542

[1.48853236e-01 1.33490097e-01 4.47348532e-02 9.80501140e-02
 2.63594586e-02 0.00000000e+00 1.97121053e-01 1.32434829e-01
 6.96051521e-02 2.23299034e-01 3.87455961e-02 0.00000000e+00
 1.06249305e-01 7.20096348e-02 2.15928696e-02 1.11265640e-01
 2.27147958e-03 0.00000000e+00 7.71059150e-02 5.31752459e-02
 1.37362155e-02 9.25344382e-02 0.00000000e+00 0.00000000e+00
 2.48793870e-02 1.59176956e-02 5.24077739e-03 2.46668366e-02
 1.00889639e-03 0.00000000e+00 2.46330402e-02 1.72679404e-02
 5.29067687e-03 2.08834147e-02 1.45171655e-03 0.00000000e+00
 8.00000000e+00 2.00000000e+00 9.00000000e+00 0.00000000e+00]
[1.46638279e-01 4.39323100e-02 5.28025858e-02 1.35899704e-01
 1.87511893e-01 0.00000000e+00 1.66861021e-01 8.07256577e-02
 3.97083830e-02 2.56969846e-01 1.38792182e-01 0.00000000e+00
 7.32576036e-02 3.46904699e-02 2.95078010e-02 1.27521939e-01
 3.99186324e-02 0.00000000e+00 5.71985716e-02 2.27385353e-02
 1.42840275e-02 1.04371359e-01 0.00000000e+00 0.00000000e+00
 2.26459893e-02 1.01522

[1.94539482e-01 1.33059869e-01 5.21125489e-02 1.06068056e-01
 3.54655381e-01 0.00000000e+00 2.04695884e-01 1.32073181e-01
 4.71942981e-02 2.28864767e-01 2.81984734e-01 9.44003872e-04
 9.16212044e-02 7.57954878e-02 2.04582412e-02 1.12716996e-01
 7.16826705e-02 0.00000000e+00 7.38700526e-02 5.58605089e-02
 1.70674742e-02 9.57537721e-02 1.13271796e-01 0.00000000e+00
 2.64838473e-02 2.06160333e-02 1.12092057e-02 2.50901418e-02
 4.52188261e-02 0.00000000e+00 2.15989894e-02 1.60749388e-02
 5.94956351e-03 2.16285541e-02 8.87362517e-03 0.00000000e+00
 1.40000000e+01 1.00000000e+00 1.10000000e+01 1.00000000e+00]
[1.80893492e-01 1.64563587e-01 3.27253004e-02 9.24666162e-02
 3.51173056e-01 0.00000000e+00 2.18421078e-01 1.48537931e-01
 2.05275123e-02 2.20427383e-01 6.20122056e-03 0.00000000e+00
 1.18289114e-01 7.33579104e-02 1.72326049e-02 1.09365044e-01
 1.64269752e-02 1.49625829e-06 9.19110331e-02 6.34740406e-02
 1.28516274e-02 9.02691179e-02 9.79108336e-02 0.00000000e+00
 2.92777968e-02 1.99913

[2.24917765e-01 1.27657221e-01 3.85616851e-02 9.24134190e-02
 9.39017505e-02 0.00000000e+00 2.24065233e-01 1.24387883e-01
 3.00205056e-02 2.18136203e-01 1.95214672e-01 1.31997985e-03
 1.06340880e-01 5.91416634e-02 1.33508382e-02 1.07838544e-01
 2.10173794e-04 0.00000000e+00 7.89945330e-02 4.57114471e-02
 1.43939292e-02 8.83850741e-02 5.03881255e-02 0.00000000e+00
 2.91184279e-02 1.87417785e-02 3.34645395e-03 2.38041683e-02
 3.45779637e-02 2.42181127e-04 2.38877696e-02 1.33246476e-02
 5.18808491e-03 1.97784356e-02 3.39397914e-02 4.73078981e-07
 4.00000000e+00 0.00000000e+00 5.00000000e+00 0.00000000e+00]
[1.93444170e-01 1.11345017e-01 6.19476718e-02 9.14838582e-02
 3.15809546e-01 0.00000000e+00 1.37439085e-01 7.63629990e-02
 5.69584041e-02 2.08151769e-01 6.47212218e-02 0.00000000e+00
 7.59136806e-02 4.47516349e-02 2.87412654e-02 1.05737405e-01
 7.85143452e-02 0.00000000e+00 6.56942070e-02 3.76071914e-02
 1.51002904e-02 9.51087552e-02 0.00000000e+00 0.00000000e+00
 2.41355875e-02 1.26546

...

[2.23548110e-01 1.77402858e-01 8.99413239e-02 9.73909121e-02
 2.81736525e-03 0.00000000e+00 2.58571638e-01 1.73371546e-01
 2.82762397e-02 2.23660195e-01 1.01201718e-01 6.63778512e-04
 1.32784786e-01 8.89577008e-02 2.10374183e-02 1.10283632e-01
 4.60060405e-02 1.37192005e-03 9.07831154e-02 5.30193595e-02
 1.39661666e-02 9.05256675e-02 1.21187756e-02 0.00000000e+00
 3.11988997e-02 1.97664557e-02 1.16150103e-02 2.50403765e-02
 1.07741511e-03 0.00000000e+00 3.01731456e-02 1.88689181e-02
 6.31465423e-03 2.05269961e-02 1.96460803e-03 2.77264032e-05
 6.00000000e+00 0.00000000e+00 8.00000000e+00 0.00000000e+00]
[0.16158443 0.06044558 0.04338392 0.11067771 0.01563935 0.
 0.18219674 0.09410007 0.04715852 0.22652298 0.25569547 0.
 0.07275232 0.0484924  0.03897993 0.11431215 0.11048701 0.
 0.06829884 0.03405696 0.00817374 0.10025981 0.0502715  0.
 0.02411576 0.0113151  0.00988522 0.02584337 0.01669815 0.
 0.02075968 0.01103411 0.01007981 0.02274626 0.         0.
 3.         0.         2.         0

[2.23052136e-01 2.07202006e-01 1.58600767e-02 8.75696816e-02
 2.18262713e-01 1.99790825e-05 2.05791952e-01 1.33322400e-01
 6.04449057e-02 2.22168999e-01 2.87762413e-01 0.00000000e+00
 1.12099451e-01 7.31220460e-02 3.39593868e-02 1.08632447e-01
 1.09311080e-01 0.00000000e+00 7.83671609e-02 7.80397857e-02
 1.00620683e-02 8.67447787e-02 9.06727155e-02 0.00000000e+00
 2.20637619e-02 2.78090047e-02 5.84435956e-03 2.32989756e-02
 3.56570496e-02 0.00000000e+00 2.35813711e-02 1.79930908e-02
 5.97539877e-03 1.99353889e-02 2.92547001e-02 0.00000000e+00
 1.80000000e+01 0.00000000e+00 5.00000000e+00 2.00000000e+00]
[2.29042993e-01 1.22817508e-01 6.75167204e-02 8.04944103e-02
 3.55687882e-01 1.15714159e-02 2.09248922e-01 1.21108390e-01
 9.56299395e-02 2.22374738e-01 2.02653676e-01 4.04903547e-03
 1.07382516e-01 6.16295572e-02 4.77391004e-02 1.05236713e-01
 1.47654250e-01 1.05390055e-02 8.36165299e-02 4.79168303e-02
 2.37543759e-02 8.33852444e-02 1.13311998e-01 8.50521223e-03
 2.94000573e-02 1.71043

[2.15603637e-01 1.40820255e-01 4.62836769e-02 9.25681129e-02
 0.00000000e+00 0.00000000e+00 2.43765376e-01 1.45564091e-01
 2.24647122e-02 2.13984529e-01 9.99781679e-02 2.48524639e-05
 1.13365650e-01 7.50927455e-02 1.80756276e-02 1.06290290e-01
 4.36321789e-05 0.00000000e+00 7.93771663e-02 5.66469693e-02
 7.63878249e-03 8.81888999e-02 0.00000000e+00 0.00000000e+00
 2.80250030e-02 1.81882052e-02 1.63687356e-02 2.36251666e-02
 2.09562697e-03 0.00000000e+00 2.49088003e-02 1.60014174e-02
 6.95421650e-03 1.99417535e-02 0.00000000e+00 0.00000000e+00
 2.00000000e+00 0.00000000e+00 6.00000000e+00 0.00000000e+00]
[2.19812885e-01 2.03052887e-01 1.17991841e-01 7.97817168e-02
 2.51622514e-01 0.00000000e+00 2.26022182e-01 1.57021555e-01
 1.09794884e-01 2.12032822e-01 8.44212650e-02 1.63285435e-03
 1.15187764e-01 8.52663573e-02 4.88069628e-02 1.03108321e-01
 8.02164997e-03 0.00000000e+00 7.50401546e-02 6.91973542e-02
 4.10257397e-02 8.77340858e-02 8.71821127e-02 0.00000000e+00
 3.09098998e-02 3.08723

[2.20389157e-01 2.15580449e-01 6.06296301e-02 9.87776414e-02
 1.58804349e-01 0.00000000e+00 2.62555634e-01 1.70294041e-01
 2.00588654e-02 2.27890863e-01 3.00959166e-01 6.70999509e-05
 1.33682972e-01 8.89827316e-02 1.17837004e-02 1.11571752e-01
 1.40888939e-01 5.60945722e-05 9.73907216e-02 6.10304905e-02
 7.29909623e-03 9.22444434e-02 1.12678180e-01 1.58293556e-02
 3.41182390e-02 2.42389770e-02 1.20385667e-02 2.51946147e-02
 3.56080356e-02 0.00000000e+00 3.00424533e-02 1.84831116e-02
 2.82464176e-03 2.08581337e-02 2.81653365e-02 5.74918190e-04
 2.30000000e+01 0.00000000e+00 9.00000000e+00 0.00000000e+00]
[1.82195618e-01 1.84411941e-01 5.72768154e-02 1.02364636e-01
 3.62809797e-03 0.00000000e+00 2.30721951e-01 1.53166127e-01
 4.79496596e-02 2.22676421e-01 9.43162373e-02 0.00000000e+00
 1.04517805e-01 8.30104403e-02 2.07440075e-02 1.10940660e-01
 0.00000000e+00 0.00000000e+00 7.00467273e-02 7.20488818e-02
 2.89665384e-02 9.07718558e-02 4.65467410e-02 0.00000000e+00
 2.91527611e-02 2.10861

KeyboardInterrupt: 